In [67]:
import numpy as np
import pandas as pd 
import geopandas as gpd 
import matplotlib.pyplot as plt 
from math import ceil
df_crime = pd.read_csv('MPS LSOA Level Crime (most recent 24 months).csv', na_values=[':'], low_memory=False)
df_lookup = pd.read_csv('2011 _OA-LSOA-MSOA-LA.csv', na_values=[':'], low_memory=False)

In [68]:
crime_column_name=df_crime.columns.values[4:27]

In [69]:
crime_sum=df_crime[df_crime.columns.values[-1]]

In [70]:
for i in crime_column_name: #add all crime number during latest 2 years.
    crime_sum=crime_sum+df_crime[i]

In [71]:
df_crime.drop(columns=crime_column_name,inplace=True)
df_crime.drop(columns=df_crime.columns.values[-1],inplace=True)

In [72]:
df_crime.drop(columns=['Borough','Minor Category'],inplace=True)

In [73]:
df_crime['Crime_sum']=crime_sum

,LSOA Code,Major Category,Crime_sum
0,E01000006,Arson and Criminal Damage,2
1,E01000007,Arson and Criminal Damage,1
2,E01000008,Arson and Criminal Damage,1
3,E01000009,Arson and Criminal Damage,2
4,E01000010,Arson and Criminal Damage,3
...,...,...,...
103837,E01033604,Violence Against the Person,147
103838,E01033605,Violence Against the Person,99
103839,E01033606,Violence Against the Person,27
103840,E01033607,Violence Against the Person,32


In [74]:
df_lookup.drop(columns=['OA11CD','LSOA11NM','MSOA11NM','LAD11CD','LAD11NM','Inner_Outer'],inplace=True)

In [75]:
df_lookup.duplicated()

0        False
1         True
2         True
3         True
4        False
         ...  
25048     True
25049     True
25050     True
25051     True
25052     True
Length: 25053, dtype: bool

In [76]:
df_lookup_no_dupli=df_lookup[~df_lookup.duplicated()]

In [77]:
df_lookup_no_dupli

,LSOA11CD,MSOA11CD
0,E01000001,E02000001
4,E01000003,E02000001
8,E01000002,E02000001
16,E01032739,E02000001
17,E01032740,E02000001
...,...,...
25021,E01001638,E02006931
25023,E01001641,E02006931
25025,E01001642,E02006931
25031,E01033746,E02006931


In [78]:
data_merged=pd.merge(left=df_crime,right=df_lookup_no_dupli,how='left',left_on='LSOA Code',right_on='LSOA11CD')

In [79]:
data_merged

,LSOA Code,Major Category,Crime_sum,LSOA11CD,MSOA11CD
0,E01000006,Arson and Criminal Damage,2,E01000006,E02000017
1,E01000007,Arson and Criminal Damage,1,E01000007,E02000016
2,E01000008,Arson and Criminal Damage,1,E01000008,E02000016
3,E01000009,Arson and Criminal Damage,2,E01000009,E02000017
4,E01000010,Arson and Criminal Damage,3,E01000010,E02000016
...,...,...,...,...,...
103837,E01033604,Violence Against the Person,147,E01033604,E02000968
103838,E01033605,Violence Against the Person,99,E01033605,E02000968
103839,E01033606,Violence Against the Person,27,E01033606,E02000973
103840,E01033607,Violence Against the Person,32,E01033607,E02000973


In [80]:
group=data_merged.groupby(['MSOA11CD','Major Category'])

In [81]:
data_group=group['Crime_sum'].agg(['sum','mean'])

In [82]:
data_group

sum       mean
MSOA11CD  Major Category                                      
E02000002 Arson and Criminal Damage             127  21.166667
          Burglary                              102  14.571429
          Drug Offences                          39   5.571429
          Miscellaneous Crimes Against Society   22   1.466667
          Possession of Weapons                   8   1.600000
...                                             ...        ...
E02006931 Public Order Offences                 148  11.384615
          Robbery                               128  14.222222
          Theft                                 611  30.550000
          Vehicle Offences                      232  13.647059
          Violence Against the Person           483  43.909091

[9812 rows x 2 columns]

In [83]:
data_pivot=pd.pivot_table(data_merged,index='MSOA11CD',columns='Major Category',aggfunc='sum')

In [84]:
#data_pivot=data_pivot.reset_index()

In [85]:
data_pivot.columns=data_pivot.columns.droplevel(0)

In [86]:
data_pivot.fillna(0,inplace=True)

In [87]:
data_pivot.columns.values

array(['Arson and Criminal Damage', 'Burglary', 'Drug Offences',
       'Miscellaneous Crimes Against Society', 'Possession of Weapons',
       'Public Order Offences', 'Robbery', 'Theft', 'Vehicle Offences',
       'Violence Against the Person'], dtype=object)

In [88]:
#housing prices data by MSOA
gdf = gpd.read_file(f"zip://statistical-gis-boundaries-london.zip!statistical-gis-boundaries-london/ESRI/MSOA_2011_London_gen_MHW.shp")
df = pd.read_csv('https://data.london.gov.uk/download/average-house-prices/bdf8eee7-41e1-4d24-90ce-93fe5cf040ae/land-registry-house-prices-MSOA.csv', na_values=[':'], low_memory=False)
print(df.shape)
measure = 'Mean'
df_years=[]
for year in range(2011,2018):
    df_temp = df[(df.Year==f'Year ending Dec {year}') & (df.Measure==measure)].copy().reset_index()
    df_temp.drop(index=df_temp[df_temp.Value.isna()].index, inplace=True)
    df_temp['Measure'] = df_temp.Measure.astype('category')
    df_temp['Value'] = df_temp.Value.astype('int')
    df_temp.drop(columns=['index'],inplace=True)
    print(df_temp.shape)
    
    column_name=list(df_temp.columns.values)
    column_name[4]=f'Value{year}'
    df_temp.columns=np.array(column_name)
    
    df_years.append(df_temp)


(262461, 5)
(983, 5)
(983, 5)
(983, 5)
(983, 5)
(983, 5)
(983, 5)
(983, 5)


In [89]:
df_years[6]

,Code,Area,Year,Measure,Value2017
0,E02000001,City of London 001,Year ending Dec 2017,Mean,950760
1,E02000002,Barking and Dagenham 001,Year ending Dec 2017,Mean,296691
2,E02000003,Barking and Dagenham 002,Year ending Dec 2017,Mean,320546
3,E02000004,Barking and Dagenham 003,Year ending Dec 2017,Mean,333292
4,E02000005,Barking and Dagenham 004,Year ending Dec 2017,Mean,323039
...,...,...,...,...,...
978,E02000979,Westminster 020,Year ending Dec 2017,Mean,1545453
979,E02000980,Westminster 021,Year ending Dec 2017,Mean,1329539
980,E02000981,Westminster 022,Year ending Dec 2017,Mean,966280
981,E02000982,Westminster 023,Year ending Dec 2017,Mean,1059011


In [90]:
for m in range(2,7):
    print(m)

2
3
4
5
6


In [91]:
df_merge=pd.merge(df_years[0],df_years[1],left_on='Code',right_on='Code',how='inner')
for i in range(2,7):
    df_merge=pd.merge(df_merge,df_years[i],left_on='Code',right_on='Code',how='inner')

In [92]:
df_merge.reset_index(inplace=True)

In [93]:
df_merge.drop(columns=['Area_x','Year_x','Measure_x','Measure','Year','Area_y','Year_y','Measure_y'],inplace=True)
df_merge

,index,Code,Value2011,Value2012,Value2013,Value2014,Value2015,Value2016,Area,Value2017
0,0,E02000001,541192,537160,775504,894046,926292,928037,City of London 001,950760
1,1,E02000002,191618,189744,189936,218348,248572,283701,Barking and Dagenham 001,296691
2,2,E02000003,193483,195934,206912,233599,263405,286872,Barking and Dagenham 002,320546
3,3,E02000004,195785,204150,206944,239080,274326,335301,Barking and Dagenham 003,333292
4,4,E02000005,168656,165630,183049,222851,256392,315574,Barking and Dagenham 004,323039
...,...,...,...,...,...,...,...,...,...,...
978,978,E02000979,1035122,1176059,1058993,1442447,1443883,1486409,Westminster 020,1545453
979,979,E02000980,636388,691133,793470,868984,1080080,1865278,Westminster 021,1329539
980,980,E02000981,814260,808404,882129,1003818,1042377,1017398,Westminster 022,966280
981,981,E02000982,827082,1068366,1204611,2328898,1470763,1336629,Westminster 023,1059011


In [94]:
#merge housing price and crime number
merge_crime_housing=pd.merge(left=df_merge,right=data_pivot,how='left',left_on='Code',right_on='MSOA11CD')

In [95]:
merge_crime_housing.fillna(0,inplace=True)

In [96]:
for i in data_pivot.columns.values:
    merge_crime_housing[i]=merge_crime_housing[i].astype(int)

In [97]:
merge_crime_housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983 entries, 0 to 982
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   index                                 983 non-null    int64 
 1   Code                                  983 non-null    object
 2   Value2011                             983 non-null    int32 
 3   Value2012                             983 non-null    int32 
 4   Value2013                             983 non-null    int32 
 5   Value2014                             983 non-null    int32 
 6   Value2015                             983 non-null    int32 
 7   Value2016                             983 non-null    int32 
 8   Area                                  983 non-null    object
 9   Value2017                             983 non-null    int32 
 10  Arson and Criminal Damage             983 non-null    int32 
 11  Burglary                        

In [98]:
ppd = pd.merge(gdf[['MSOA11CD','geometry']], merge_crime_housing, left_on='MSOA11CD', right_on='Code', how='inner').reset_index()
print(ppd.shape)

(983, 23)


In [99]:
ppd.drop(columns=['level_0','index','MSOA11CD'],inplace=True)

In [100]:
ppd

,geometry,Code,Value2011,Value2012,Value2013,Value2014,Value2015,Value2016,Area,Value2017,Arson and Criminal Damage,Burglary,Drug Offences,Miscellaneous Crimes Against Society,Possession of Weapons,Public Order Offences,Robbery,Theft,Vehicle Offences,Violence Against the Person
0,"MULTIPOLYGON (((531667.624 180534.992, 531647....",E02000001,541192,537160,775504,894046,926292,928037,City of London 001,950760,0,0,0,0,0,0,0,0,0,0
1,"POLYGON ((548881.563 190845.265, 548881.125 19...",E02000002,191618,189744,189936,218348,248572,283701,Barking and Dagenham 001,296691,127,102,39,22,8,91,39,127,209,481
2,"POLYGON ((549102.438 189324.625, 548954.500 18...",E02000003,193483,195934,206912,233599,263405,286872,Barking and Dagenham 002,320546,157,158,117,30,18,123,96,301,306,579
3,"POLYGON ((551549.998 187364.637, 551478.000 18...",E02000004,195785,204150,206944,239080,274326,335301,Barking and Dagenham 003,333292,53,43,50,18,9,48,22,85,84,245
4,"POLYGON ((549099.634 187656.076, 549161.375 18...",E02000005,168656,165630,183049,222851,256392,315574,Barking and Dagenham 004,323039,119,115,44,22,8,109,62,158,177,485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,"POLYGON ((541029.881 176265.023, 541101.997 17...",E02006927,382473,323093,384225,459465,468814,504756,Greenwich 034,497229,90,175,59,17,17,110,68,170,296,456
979,"MULTIPOLYGON (((538300.370 177956.632, 538290....",E02006928,313753,307449,433378,462623,483581,494008,Greenwich 035,491609,150,225,106,19,11,146,108,569,169,672
980,"POLYGON ((540054.366 179353.136, 540220.352 17...",E02006929,310388,343855,355856,477567,422423,496217,Greenwich 036,548934,289,313,296,40,30,255,150,1343,614,995
981,"POLYGON ((539552.478 178604.042, 539533.000 17...",E02006930,396548,403596,421069,386279,491540,581988,Greenwich 037,513752,129,170,93,23,12,167,62,580,296,494


In [101]:
ppd.crs

<Projected CRS: PROJCS["OSGB 1936 / British National Grid",GEOGCS[ ...>
Name: OSGB 1936 / British National Grid
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: OSGB 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [102]:
ppd.to_file('./output/London_Housing_crime.shp',
           driver='ESRI Shapefile',
           encoding='utf-8')

In [103]:
ppd.mean()

Value2011                               372641.584944
Value2012                               396649.773143
Value2013                               431201.346897
Value2014                               499355.365209
Value2015                               536904.321465
Value2016                               577512.311292
Value2017                               603715.492370
Arson and Criminal Damage                  105.689725
Burglary                                   135.376399
Drug Offences                              104.216684
Miscellaneous Crimes Against Society        21.968464
Possession of Weapons                       13.135300
Public Order Offences                      106.954222
Robbery                                     62.781282
Theft                                      396.088505
Vehicle Offences                           230.894201
Violence Against the Person                439.359105
dtype: float64